**The two blocks below contain the code to create inward2.csv and outward2.csv two files which have incoming transactions and outgoing transactions.**

They are then merged to form **coinlog.csv**
MAy want run the following:

*coinlog['Date'] = pd.to_datetime(coinlog['Date'])*

The price and transaction data are merged and is available at **price_trans.csv**

In [4]:
import pandas as pd
price_trans = pd.read_csv("price_trans.csv")
coinlog = pd.read_csv("coinlog.csv",  low_memory= False ) 
coinlog['Date'] = pd.to_datetime(coinlog['Date'])
price_trans['Date'] = pd.to_datetime(price_trans['Date'])

## Splitting
outgoing = coinlog.query('outquals1 == 1')
incoming = coinlog.query('outquals1 == 0') 

In [5]:
## Piping and new small dataframes

## Getting Transaction Count
trans_bydate = outgoing.groupby('Date', as_index = False).agg({"txID": "count"})
# trans_bydate.to_csv("trans_bydate.csv", encoding ='utf-8', index=False) 

#trans_bydate = outgoing.groupby('Date').count()
#trans_bydate.head()

## Getting Volume 
vol_bydate = outgoing.groupby('Date', as_index = False).agg({"Satoshi": "sum"})

#vol_bydate.head()

## Getting number of addresses

#ADD_bydate = coinlog.groupby('Date', as_index = False).agg({"rec_ADD": "count"})

#ADD_bydate.head()


## Getting average transaction size by data

avgsize_bydate = outgoing.groupby('Date', as_index = False).agg({"Satoshi": "mean"})

#avgsize_bydate.head()

In [9]:
## Getting weighted time to execution

#coinlog.set_index(['txID', 'outquals1'])

wait_bytxmin = coinlog.groupby('txID', as_index = False).agg({"Time": "min"})
wait_bytxmax = coinlog.groupby('txID', as_index = False).agg({"Time": "max"}) 

#coinlog.head()
#coinlog['Time'] = pd.to_datetime(coinlog['Time'])
#coinlog['Time'] = pd.to_numeric(coinlog['Time'])
#coinlog.dtypes

In [10]:
wait_bytxmin.columns = ['txID','tmin']
wait_bytxmax.columns = ['txID','tmax']
wait_bytx = pd.merge(wait_bytxmin, wait_bytxmax, on='txID', how='left')

wait_bytx['tdiff'] = wait_bytx['tmax'] - wait_bytx['tmin']

wait_bytx.head()

wait_bytx.to_csv("txwait.csv", encoding= 'utf-8', index=False)

In [12]:
## Merge with outgoing

#outgoing_delta = pd.merge(outgoing, wait_bytx, on='txID', how='left')
outgoing_delta['tdiff'] = outgoing_delta['tdiff']/1000000000
outgoing_delta.head()

,txID,rec_ADD,Satoshi,User,Time,Date,outquals1,tmin,tmax,tdiff
0,171,87947,5000000000,87947.0,2009-01-12 03:30:25,2009-01-12,1,1231731025000000000,1233973867000000000,2242842.0
1,183,87947,4000000000,87947.0,2009-01-12 06:02:13,2009-01-12,1,1231740133000000000,1233975266000000000,2235133.0
2,185,87947,3000000000,87947.0,2009-01-12 06:12:16,2009-01-12,1,1231740736000000000,1234108300000000000,2367564.0
3,187,87947,2900000000,87947.0,2009-01-12 06:34:22,2009-01-12,1,1231742062000000000,1234200634000000000,2458572.0
4,192,412501,100000000,412501.0,2009-01-12 07:16:40,2009-01-12,1,1231744600000000000,1234200634000000000,2456034.0


In [15]:
## Average tdiff by date 

meantdiff_bydate = outgoing_delta.groupby('Date', as_index = False).agg({"tdiff": "mean"})
vartdiff_bydate = outgoing_delta.groupby('Date', as_index = False).agg({"tdiff": "var"})

In [17]:
meantdiff_bydate.columns = ["Date","mean_tdiff"]
vartdiff_bydate.columns = ["Date", "var_tdiff"]
meantdiff_bydate.head()

,Date,mean_tdiff
0,2009-01-12,2.458673e+06
1,2009-01-14,5.602298e+06
2,2009-01-15,6.562183e+06
3,2009-01-16,8.135062e+06
4,2009-01-18,1.338550e+07


In [56]:
k.columns = ["Date"]
k.head()

0   2009-01-12
1   2009-01-14
2   2009-01-15
3   2009-01-16
4   2009-01-18
dtype: datetime64[ns]

In [18]:
#p = pd.DataFrame()
#p['Date'] = outgoing['Date'].unique()
#p['#tx'] = trans_bydate['txID']
#p['Vol'] = vol_bydate['Satoshi']
#p['mean_tx'] = avgsize_bydate['Satoshi']

p['mean_tdiff'] = meantdiff_bydate['mean_tdiff']
p['var_tdiff'] = vartdiff_bydate['var_tdiff']

In [19]:
p.head()

,Date,#tx,Vol,mean_tx,mean_tdiff,var_tdiff
0,2009-01-12,7,17900000000,2.557143e+09,2.458673e+06,7.012197e+10
1,2009-01-14,3,6100000000,2.033333e+09,5.602298e+06,0.000000e+00
2,2009-01-15,15,50000000000,3.333333e+09,6.562183e+06,1.311762e+11
3,2009-01-16,4,20000000000,5.000000e+09,8.135062e+06,1.281710e+08
4,2009-01-18,3,15000000000,5.000000e+09,1.338550e+07,0.000000e+00


In [23]:
price_trans['Date'].dtype

dtype('<M8[ns]')

In [21]:
price_trans.head()

,Date,Transaction,Open_price,High_price,Low_Price,Close_Price,Ex_volume,Wt_Price
0,2013-12-28,11.802597,803.23000,806.00000,715.60,762.005,13149.183080,758.732555
1,2013-12-27,13.521354,807.00000,837.00000,768.00,803.230,13466.884860,798.439412
2,2013-12-26,11.777376,707.30000,829.80000,707.30,802.110,23193.336990,774.102778
3,2013-12-25,12.457998,696.86000,710.00000,674.00,707.300,5270.695376,696.187775
4,2013-12-24,12.645341,716.99999,729.99174,666.01,702.800,10677.145200,698.733036


In [28]:
p['Date'].min()
#p.corr()

Timestamp('2009-01-12 00:00:00')

In [29]:
workingframe = pd.merge(p, price_trans, on='Date', how='left')

In [41]:
workingframe2.to_csv("working2.csv", encoding= 'utf-8', index=False)

In [37]:
import numpy as np
workingframe2 = workingframe[np.isfinite(workingframe['Open_price'])]

In [40]:
workingframe2.describe()

,#tx,Vol,mean_tx,mean_tdiff,var_tdiff,Transaction,Open_price,High_price,Low_Price,Close_Price,Ex_volume,Wt_Price
count,1261.000000,1.261000e+03,1.261000e+03,1.261000e+03,1.261000e+03,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000
mean,52079.803331,1.313686e+14,6.001398e+09,-2.746948e+08,7.071122e+15,19.935286,59.389069,63.458982,54.965682,60.010909,43035.724493,59.264800
std,52590.516539,2.371450e+14,9.425218e+09,1.450783e+09,2.185792e+17,24.381121,160.465787,173.221327,144.763693,161.775835,48000.295834,158.926773
min,168.000000,8.604590e+11,3.125891e+08,-7.838769e+09,4.711160e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8961.000000,2.800630e+13,1.320953e+09,2.596465e+06,1.168353e+08,3.499371,2.770000,2.981000,2.500000,2.773020,14637.479060,2.791256
50%,18970.000000,7.176332e+13,3.059287e+09,3.427822e+06,3.196501e+08,14.433946,7.611010,8.359940,7.055000,7.665100,30176.739000,7.629051
75%,96096.000000,1.483244e+14,6.207535e+09,5.447418e+06,1.407946e+09,30.275889,27.249970,29.399000,26.151000,27.220500,54344.491070,27.164364
max,255956.000000,3.137447e+15,9.569929e+10,9.470580e+06,7.705559e+18,305.202135,1230.005000,1242.000000,1150.250000,1237.955000,572185.685200,1209.937320


In [68]:
type(outgoing.groupby('Date').txID.agg(['min','count']))

pandas.core.frame.DataFrame

In [36]:
type(trans_bydate['count'])

pandas.core.series.Series

In [20]:
p.to_csv("daywise.csv", encoding= 'utf-8', index=False)

In [ ]:
#receive_bydate = outgoing.groupby('Date').txID.count()

In [ ]:
import pandas as pd

## For receiving accounts
# Read from txin
data = pd.read_csv('txin.txt', sep='\t', header = None) 

# Give column names
data.columns = ["txID", "rec_ADD", "Satoshi"]

# Read from contraction for user mapping
data2 = pd.read_csv('contraction.txt', sep='\t')

# Merge the two
receive = pd.merge(data, data2, on='rec_ADD', how='left')

# Read in transaction time
data3 = pd.read_csv('txtime.txt', sep='\t', header = None) 

# Give it column names
data3.columns = ['txID' ,'Time']

# Merging transaction times 
receive1 = pd.merge(receive, data3, on='txID', how='left')

# Getting time to a better format
receive1['Time'] = pd.to_datetime(receive1['Time'], unit='s')
receive1['Date'] = receive1['Time'].dt.date

In [ ]:
## For sending accounts

# Read from txout
outward = pd.read_csv('txout.txt', sep='\t', header = None) 

# Give column names
outward.columns = ["txID", "rec_ADD", "Satoshi"]

# Merge the two
outward = pd.merge(outward, data2, on='rec_ADD', how='left')

# Merging transaction times with outward
outward1 = pd.merge(outward, data3, on='txID', how='left')

# Getting a better time representation

outward1['Time'] = pd.to_datetime(receive1['Time'], unit='s')
outward1['Date'] = receive1['Time'].dt.date


**Reading in prepared inbound and outbound transaction data**

In [16]:
#inward = pd.read_csv("inward2.csv")
outward = pd.read_csv("outward2.csv")

/home/ngeorge1/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
inward['inequals1'] = 1
inward.head()

,txID,rec_ADD,Satoshi,User,Time,Date,inequals1
0,171,87947,5000000000,87947,2009-01-12 03:30:25,2009-01-12,1
1,183,87947,4000000000,87947,2009-01-12 06:02:13,2009-01-12,1
2,185,87947,3000000000,87947,2009-01-12 06:12:16,2009-01-12,1
3,187,87947,2900000000,87947,2009-01-12 06:34:22,2009-01-12,1
4,192,412501,100000000,412501,2009-01-12 07:16:40,2009-01-12,1


In [19]:
outward['inequals1'] = 0
outward.head()


,txID,rec_ADD,Satoshi,User,Time,Date,inequals1
0,0,2257184,5000000000,2257184.0,2009-01-12 03:30:25,2009-01-12,0
1,1,86812,5000000000,86812.0,2009-01-12 06:02:13,2009-01-12,0
2,2,4421818,5000000000,4421818.0,2009-01-12 06:12:16,2009-01-12,0
3,3,3943916,5000000000,3943916.0,2009-01-12 06:34:22,2009-01-12,0
4,4,1072333,5000000000,1072333.0,2009-01-12 07:16:40,2009-01-12,0


**Data Processing Messiness of getting the pricing together**

In [40]:
#import pandas as pd
#price = pd.read_csv("Price.csv")
#transaction = pd.read_csv("Transaction.csv")

#price_trans = pd.merge(transaction, price, on='Date', how='left')
#del price_trans['Volume ']
#del price_trans['Volume']
price_trans.columns = ["Date", "Transaction", "Open_price", "High_price", "Low_Price", "Close_Price","Ex_volume", "Wt_Price"]
# list(price_trans)

In [41]:
price_trans.head()

,Date,Transaction,Open_price,High_price,Low_Price,Close_Price,Ex_volume,Wt_Price
0,12/28/2013,11.802597,803.23000,806.00000,715.60,762.005,13149.183080,758.732555
1,12/27/2013,13.521354,807.00000,837.00000,768.00,803.230,13466.884860,798.439412
2,12/26/2013,11.777376,707.30000,829.80000,707.30,802.110,23193.336990,774.102778
3,12/25/2013,12.457998,696.86000,710.00000,674.00,707.300,5270.695376,696.187775
4,12/24/2013,12.645341,716.99999,729.99174,666.01,702.800,10677.145200,698.733036


In [3]:
transaction = pd.read_csv("Transaction.csv")

In [12]:
transaction.head()

,Date,Value
0,12/28/2013,11.802597
1,12/27/2013,13.521354
2,12/26/2013,11.777376
3,12/25/2013,12.457998
4,12/24/2013,12.645341


In [30]:
price_trans = pd.merge(transaction, price, on='Date', how='left')
del price_trans['Volume ']
price_trans = ["Date", "Transaction", "Open_price", "High_price", "Low_Price", "Close_Price", "Wt_Price"]
# list(price_trans)

In [31]:
price_trans.head()


AttributeError: 'list' object has no attribute 'head'

In [20]:
coinlog =  pd.concat([inward, outward])

,txID,rec_ADD,Satoshi,User,Time,Date,inequals1
0,171,87947,5000000000,87947.0,2009-01-12 03:30:25,2009-01-12,1
1,183,87947,4000000000,87947.0,2009-01-12 06:02:13,2009-01-12,1
2,185,87947,3000000000,87947.0,2009-01-12 06:12:16,2009-01-12,1
3,187,87947,2900000000,87947.0,2009-01-12 06:34:22,2009-01-12,1
4,192,412501,100000000,412501.0,2009-01-12 07:16:40,2009-01-12,1


In [38]:
#price_trans.head()
price_trans.shape

(1821, 8)

In [67]:
price_trans.loc[price_trans['Date'] == '10/27/2013']

,Date,Transaction,Open_price,High_price,Low_Price,Close_Price,Ex_volume,Wt_Price
62,2013-10-27,44.034456,189.92363,207.9,189.0,207.0001,12904.93484,199.821354


In [70]:
coinlog.loc[coinlog['txID'] == 26068509]

,txID,rec_ADD,Satoshi,User,Time,Date,outquals1
55930422,26068509,23865869,389761,23865869.0,2013-10-27 00:01:40,2013-10-27,1
55930423,26068509,24391925,3003154768,23865869.0,2013-10-27 00:01:40,2013-10-27,1
128982463,26068509,21457922,10000000,20352774.0,2013-12-12 17:54:23,2013-12-12,0
128982464,26068509,23046862,2993154768,23046862.0,2013-12-12 14:25:34,2013-12-12,0
128982465,26068509,23298135,289761,21647241.0,2013-12-12 14:31:00,2013-12-12,0


In [ ]:
coinlog.loc[coinlog['Date'] == '2009-04-21'] 

In [2]:
coinlog.loc[coinlog['txID'] == 23542]

,txID,rec_ADD,Satoshi,User,Time,Date,outquals1
1455,23542,896466,5000000000,896466.0,2009-09-21 19:13:19,2009-09-21,1
1456,23542,994493,5000000000,896466.0,2009-09-21 19:13:19,2009-09-21,1
1457,23542,1657273,5000000000,896466.0,2009-09-21 19:13:19,2009-09-21,1
1458,23542,2279154,5000000000,896466.0,2009-09-21 19:13:19,2009-09-21,1
1459,23542,5356255,5000000000,896466.0,2009-09-21 19:13:19,2009-09-21,1
1460,23542,5688949,5000000000,896466.0,2009-09-21 19:13:19,2009-09-21,1
1461,23542,6212366,5000000000,896466.0,2009-09-21 19:13:19,2009-09-21,1
1462,23542,6223113,5000000000,896466.0,2009-09-21 19:13:19,2009-09-21,1
65737823,23542,5751459,40000000000,1761.0,2010-06-13 11:06:53,2010-06-13,0


In [7]:
#coinlog.columns = ["txID", "rec_ADD", "Satoshi", "User", "Time", "Date","outquals1"]
coinlog['User'] = pd.to_numeric(coinlog['User'])

In [13]:
#price_trans.to_csv("price_trans.csv", encoding='utf-8', index=False)
#import pandas as pd
#price_trans = pd.read_csv("price_trans.csv")

#coinlog.to_csv("coinlog.csv", encoding= 'utf-8', index=False)

#coinlog = pd.read_csv("coinlog.csv") 

outgoing = coinlog.query('outquals1 == 1')
incoming = coinlog.query('outquals1 == 0') 

In [9]:
incoming.head()

,txID,rec_ADD,Satoshi,User,Time,Date,outquals1
65714232,0,2257184,5000000000,2257184.0,2009-01-12 03:30:25,2009-01-12,0
65714233,1,86812,5000000000,86812.0,2009-01-12 06:02:13,2009-01-12,0
65714234,2,4421818,5000000000,4421818.0,2009-01-12 06:12:16,2009-01-12,0
65714235,3,3943916,5000000000,3943916.0,2009-01-12 06:34:22,2009-01-12,0
65714236,4,1072333,5000000000,1072333.0,2009-01-12 07:16:40,2009-01-12,0


In [55]:
#price_trans['Date'] = pd.to_datetime(price_trans['Date'])
#price_trans['Date'].max() - price_trans['Date'].min()

#price_trans['Date'].min() 

#coinlog['Date'] = pd.to_datetime(coinlog['Date']) 
#coinlog['Date'].max() 
#coinlog['Date'].min()

Timestamp('2009-01-12 00:00:00')

In [12]:
coinlog['Date'] = pd.to_datetime(coinlog['Date'])

In [51]:
price_trans['Date'] = pd.to_datetime(price_trans['Date'])

In [14]:
 trans_bydate = outgoing.groupby('Date').txID.count()

In [15]:
trans_bydate.head()

Date
2009-01-12     7
2009-01-14     3
2009-01-15    15
2009-01-16     4
2009-01-18     3
Name: txID, dtype: int64